In [1]:
# Day 1
# Part one
import numpy as np
data = np.loadtxt('day1.txt', dtype=int)
for j in range(2):
    data[:,j] = sorted(data[:,j])
print(np.sum(np.abs(data[:,1] - data[:,0])))

# Part two
from collections import defaultdict
numbers, counts = np.unique(data[:,1], return_counts=True)
look_up_dict = defaultdict(int)
look_up_dict.update({number:count for number, count in zip(numbers, counts)})
print(sum(d*look_up_dict[d] for d in data[:,0]))

1834060
21607792


In [2]:
# Day 2
# Part one
with open('day2.txt', 'r') as fid:
    data = [[int(d) for d in line.split()] for line in fid.read().strip().split('\n')]

import numpy as np
def is_safe(some_list):
    if len(some_list) < 1:
        return True
    current_sign = np.sign(some_list[1] - some_list[0])
    if current_sign == 0:
        return False
    for j in range(1, len(some_list)):
        diff = some_list[j] - some_list[j - 1]
        if np.sign(diff) != current_sign or abs(diff) > 3:
            return False
    return True

print(sum(is_safe(l) for l in data))

#Part two
def is_safe_when_removing_one(some_list):
    if is_safe(some_list) == True:
        return True
    for j in range(len(some_list)):
        test_list = some_list[:j] + some_list[(j + 1):]
        if is_safe(test_list) == True:
            return True
    return False
print(sum(is_safe_when_removing_one(l) for l in data))

359
418
